# Applied Data Science Capstone Week 4


## Introduction/Business Problem

In [48]:
from IPython.display import Image
Image(url= "60c172a78cd90.image.jpg",  width=800, height=800)

## Data section

#### Import Libraries 

In [49]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Define Foursquare Credentials and Version

In [50]:
CLIENT_ID = 'ICXVSITQ5UMI32QUFD1ZE0AMTFLK3ATJAB5EUB53IA3IR0XT' # your Foursquare ID
CLIENT_SECRET = 'ISWXCRS4QH3XWZQ15I0K4YVE4CIWESN5LSCYH5Q20UOL0KUC' # your Foursquare Secret
ACCESS_TOKEN = 'XQ2NBDFQXW2A5S5KOFNPUPTZ1Z43YWHDW5JRK5NMJMWZW04D' # your FourSquare Access Token
VERSION = '20210620' # current date
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ICXVSITQ5UMI32QUFD1ZE0AMTFLK3ATJAB5EUB53IA3IR0XT
CLIENT_SECRET:ISWXCRS4QH3XWZQ15I0K4YVE4CIWESN5LSCYH5Q20UOL0KUC


#### Get Manhattan Neighborhood Data

In [51]:
# load propertyshark neighborhoods data
propertyshark_data = pd.read_csv('manhattan_real_estate.csv')
propertyshark_data.head()

,Neighborhood,Borough,Median Sale Price
0,Battery Park City,Manhattan,762900.0
1,Carnegie Hill,Manhattan,1500000.0
2,Central Midtown,Manhattan,830000.0
3,Central Park South,Manhattan,2147500.0
4,Chelsea,Manhattan,1077600.0


#### Get Manhattan Neighborhood Data

In [52]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude', 'Meidian Sale Price'] 
geolocator = Nominatim(user_agent="foursquare_agent")

# instantiate the dataframe
manhattan_data = pd.DataFrame(columns=column_names)

for index, row in propertyshark_data.iterrows():
    address = row.Neighborhood + ', ' + row.Borough
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    
    manhattan_data = manhattan_data.append({'Borough': row.Borough,
                                          'Neighborhood': row.Neighborhood,
                                          'Latitude': latitude,
                                          'Longitude': longitude,
                                          'Meidian Sale Price': row[-1]}, ignore_index=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude,Meidian Sale Price
0,Manhattan,Battery Park City,40.711017,-74.016937,762900.0
1,Manhattan,Carnegie Hill,40.784197,-73.954339,1500000.0
2,Manhattan,Central Midtown,40.756182,-73.981039,830000.0
3,Manhattan,Central Park South,40.767814,-73.981232,2147500.0
4,Manhattan,Chelsea,40.746491,-74.001528,1077600.0


#### Get restaurant raw data

In [53]:
def getNearbyRestaurant(names, latitudes, longitudes, radius=500, search_query = 'Restaurant'):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, lat, lng, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        df = json_normalize(results)
        for index, row in df.iterrows():
            if row['categories']==[]:
                df.drop(index, inplace=True)
        df.reset_index(drop=True)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            row['name'], 
            row['location.lat'], 
            row['location.lng'],  
            row['categories'][0]['name']) for index, row in df.iterrows()])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [54]:
manhattan_restaurant = getNearbyRestaurant(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Battery Park City
Carnegie Hill


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]


Central Midtown
Central Park South
Chelsea
Chinatown
Civic Center
Clinton - Hell's Kitchen
East Harlem
East Village
Financial District
Flatiron District
Garment District
Gramercy Park
Greenwich Village
Harlem
Hudson Square
Hudson Yards
Inwood
Kips Bay
Koreatown
Lenox Hill
Lincoln Square
Little Italy
Lower East Side
Manhattan Valley
Morningside Heights
Murray Hill
NoHo
NoLIta
NoMad
Roosevelt Island
SoHo
Sutton Place
Theatre District - Times Square
TriBeCa
Tudor City
Turtle Bay
Two Bridges
Upper West Side
Washington Heights
West Village
Yorkville


In [55]:
print('There are {} uniques categories.'.format(len(manhattan_restaurant['Venue Category'].unique())))
manhattan_restaurant['Venue Category'].unique()

There are 125 uniques categories.


array(['Pub', 'Chinese Restaurant', 'Lounge', 'Seafood Restaurant',
       'Food Court', 'Food', 'Non-Profit', 'Sandwich Place',
       'New American Restaurant', 'Bar', 'Mediterranean Restaurant',
       'Hotel Bar', 'Diner', 'Mexican Restaurant', 'Indian Restaurant',
       'Italian Restaurant', 'American Restaurant',
       'Indian Chinese Restaurant', 'Pizza Place',
       'Latin American Restaurant', 'Asian Restaurant', 'Restaurant',
       'Coffee Shop', 'French Restaurant', 'Café', 'Sushi Restaurant',
       'Bakery', 'Sports Bar', 'Japanese Restaurant', 'Irish Pub',
       'Office', 'Hawaiian Restaurant', 'Brazilian Restaurant',
       'Event Space', 'Korean Restaurant', 'Caribbean Restaurant',
       'Vietnamese Restaurant', 'Turkish Restaurant', 'Art Museum',
       'Thai Restaurant', 'Greek Restaurant', 'Steakhouse', 'Gastropub',
       'Tapas Restaurant', 'Tech Startup', 'BBQ Joint', 'Hot Dog Joint',
       'Deli / Bodega', 'Dim Sum Restaurant', 'Cantonese Restaurant',
    

#### Data Cleaning

In [56]:
for index, row in manhattan_restaurant.iterrows():
    if row['Venue Category'] in ('Miscellaneous Shop',
                                 'Business Service',
                                 'Metro Station',
                                 'Music Venue',
                                 'Neighborhood',
                                 'Hotel',
                                 'Gift Shop',
                                 'Brewery',
                                 'Sports Bar',
                                 'Salon / Barbershop',
                                 'Office',
                                 'Art Museum',
                                 'Event Space',
                                 'Other Great Outdoors',
                                 'Conference Room',
                                 'Theater',
                                 'Building',
                                 'Gastropub','Tech Startup',
                                 'Kitchen Supply Store',
                                 'Art Gallery',
                                 'Arts & Crafts Store',
                                 'Department Store',
                                 'Housing Development',
                                 'Garden','Home Service',
                                 'Furniture / Home Store',
                                 'Non-Profit',
                                 'Hardware Store'):
        manhattan_restaurant.drop(index, inplace=True)
        manhattan_restaurant.reset_index(drop=True)

In [57]:
manhattan_restaurant['New Category'] = ''
for index, row in manhattan_restaurant.iterrows():
    if row['Venue Category'] in ('Chinese Restaurant',
                                 'Dim Sum Restaurant',
                                 'Cantonese Restaurant',
                                 'Noodle House',
                                 'Dumpling Restaurant',
                                 'Indian Chinese Restaurant',
                                 'Hotpot Restaurant',
                                 'Szechuan Restaurant'):
        manhattan_restaurant.loc[index, 'New Category'] = 'Chinese Restaurant'
    elif row['Venue Category'] in ('Thai Restaurant',
                                 'Korean Restaurant',
                                 'Japanese Restaurant',
                                 'Sushi Restaurant',
                                 'Vietnamese Restaurant',
                                 'Ramen Restaurant',
                                 'Asian Restaurant'):
        manhattan_restaurant.loc[index, 'New Category'] = 'Alternative'
    else:
        manhattan_restaurant.loc[index, 'New Category'] = 'Other'
    

manhattan_restaurant.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,New Category
0,Battery Park City,40.711017,-74.016937,O'Hara's Restaurant & Pub,40.709894,-74.012836,Pub,Other
1,Battery Park City,40.711017,-74.016937,Su Hang Restaurant,40.708363,-74.014046,Chinese Restaurant,Chinese Restaurant
2,Battery Park City,40.711017,-74.016937,Romi Restaurant & Lounge,40.707970,-74.013779,Lounge,Other
3,Battery Park City,40.711017,-74.016937,Miramar Seafood Restaurant,40.707919,-74.018159,Seafood Restaurant,Other
4,Battery Park City,40.711017,-74.016937,World Financial Center Restaurant Showcase,40.712434,-74.016497,Food Court,Other


#### Grouping and Final Table Construction

In [58]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_restaurant[['New Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_restaurant['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Alternative,Chinese Restaurant,Other
0,Battery Park City,0.000000,0.086957,0.913043
1,Carnegie Hill,0.120000,0.040000,0.840000
2,Central Midtown,0.022727,0.000000,0.977273
3,Central Park South,0.148936,0.021277,0.829787
4,Chelsea,0.146341,0.146341,0.707317


In [59]:
restaurant_count = manhattan_restaurant.groupby('Neighborhood').count()
restaurant_count = restaurant_count.Venue.to_frame()
restaurant_count.head()

,Venue
Neighborhood,
Battery Park City,23
Carnegie Hill,25
Central Midtown,44
Central Park South,47
Chelsea,41


In [60]:
temp1 = manhattan_data.merge(restaurant_count, how='inner', on='Neighborhood')
final_data = temp1.merge(manhattan_grouped, how='inner', on='Neighborhood')
final_data

,Borough,Neighborhood,Latitude,Longitude,Meidian Sale Price,Venue,Alternative,Chinese Restaurant,Other
0,Manhattan,Battery Park City,40.711017,-74.016937,762900.0,23,0.000000,0.086957,0.913043
1,Manhattan,Carnegie Hill,40.784197,-73.954339,1500000.0,25,0.120000,0.040000,0.840000
2,Manhattan,Central Midtown,40.756182,-73.981039,830000.0,44,0.022727,0.000000,0.977273
3,Manhattan,Central Park South,40.767814,-73.981232,2147500.0,47,0.148936,0.021277,0.829787
4,Manhattan,Chelsea,40.746491,-74.001528,1077600.0,41,0.146341,0.146341,0.707317
5,Manhattan,Chinatown,40.716491,-73.996250,1733179.0,46,0.108696,0.652174,0.239130
6,Manhattan,Civic Center,40.713679,-74.002404,2760000.0,49,0.204082,0.387755,0.408163
7,Manhattan,Clinton - Hell's Kitchen,40.768228,-73.994269,779500.0,16,0.000000,0.000000,1.000000
8,Manhattan,East Harlem,40.794722,-73.942500,790000.0,47,0.000000,0.127660,0.872340
9,Manhattan,East Village,40.729269,-73.987361,1049500.0,45,0.177778,0.000000,0.822222
